In [4]:
import os
import json
import pandas as pd
while os.getcwd() != '/home/jupyter/crisp':
    os.chdir("..")
%pwd

'/home/jupyter/crisp'

In [5]:
from main import run
from nasa_mice.functions import generate_mouse_human_experiment

In [6]:
verbose = 0
output_data_regime = 'binary' #'real-valued'

In [7]:
data = pd.read_pickle('data/cosmic_rads/human_mice_data_combined.pkl')

In [8]:
data['Characteristics[organism]'].value_counts()

Mus musculus    41
Homo sapiens    21
Name: Characteristics[organism], dtype: int64

In [9]:
max_humans = 21
max_mice = 41

In [10]:
for mice in range(0,41,2):
    generate_mouse_human_experiment(filepath='data/cosmic_rads/human_mice_data_combined.pkl',
                                    human_count = max_humans,
                                    mice_count = mice,
                                    seed = 42,
                                    pickle_directory = 'data/cosmic_rads/vary_mice',
                                    organism_column = 'Characteristics[organism]')

In [11]:
for humans in range(0,max_humans,1):
    generate_mouse_human_experiment(filepath='data/cosmic_rads/human_mice_data_combined.pkl',
                                    human_count = humans,
                                    mice_count = max_mice,
                                    seed = 42,
                                    pickle_directory = 'data/cosmic_rads/vary_humans',
                                    organism_column = 'Characteristics[organism]')

In [ ]:
for experiment in os.listdir('data/cosmic_rads/vary_humans'):
    if experiment.endswith('.pkl'):
        print(experiment)
        merged_data_columns = pd.read_pickle('data/cosmic_rads/vary_humans/'+experiment).columns
        try:
            os.mkdir('results/cosmic_rads/vary_humans/'+experiment[:-4]+'/')
        except Exception as e:
            print(e)

        # define config
        config = {
            "name": "Example Experiment for AH casual ensemble",
            "short_name": "ah_experiment_notebook",
            "bucket_project": "fdl-us-astronaut-health",
            "bucket_name": "ah_21_data",
            "bucket_path": "gs://ah_21_data",
            "verbose": verbose,
            "test_val_split": [0.1, 0.1],
            "per_variant_experiment": False,
            "data_options": {
                'synthetic_train_test_split' : False,
                'dataset_fp' : 'data/cosmic_rads/vary_humans/'+experiment,
                'output_data_regime' : output_data_regime,
                'subject_keys': 'Source Name',
                'targets': ['irradiated'], #['dose_Gy'],
                'predictors': list(data.columns[43:-4]),
                'environments': ['environment'],
                'exclude': ['organism', 'source', 'gender', 'age_years', 'timepoint_hr', 'dose_Gy', 'radiation', 'dataset','strain']
            },
            "feature_selection_options": {
                "max_features": 20,
                "verbose": verbose,
                "seed": 12,
                "variance_threshold" : 2, # actually std
            },
            "ensemble_options": {
                "models": ["IRM","LIRM"],#, "ICP",], # "DCF", "LIRM", "NLICP"],
                "LIRM" : {
                    "use_icp_initialization": False,
                    "verbose": verbose,
                    "n_iterations": 1000,
                    "seed": 0,
                    "lr": 0.001,
                    "cuda": False,
                    "output_data_regime" : output_data_regime,
                },
                "DCF" : {
                    "minP": 0.1,
                    "maxP": 0.9,
                    "minFeatures": 1,
                    "minAccuracy": 0.5,
                    "seed": 0,
                    "verbose": verbose,
                    "target": ["Target"],
                    "output_pvals": False,
                    "output_data_regime" : output_data_regime,
                },
                "NLICP" : {
                    "max_set_size": 2,
                    "alpha": 0.05,
                    "seed": 12,
                    "verbose": verbose,
                    "method": "MLP",
                    "hidden_dim": 256,
                    "output_data_regime" : output_data_regime,
                },
                "LICP" : {
                    "max_set_size": 2,
                    "alpha": 0.05,
                    "seed": 12,
                    "verbose": verbose,
                    "output_data_regime" : output_data_regime,
                },
            },
             "use_cloud":False,
            "results_directory": "results/cosmic_rads/vary_humans/"+experiment[:-4]+'/'
            }
        print(config['results_directory'])
        run(config)